In [ ]:
import ROOT
import sys

In [1]:
import ROOT
import sys
import os

def create_mini_root_file(input_file_path, output_file_path, branch_list):
    """
    Create a smaller ROOT file containing only specified branches from a larger ROOT file.
    Added debugging information and progress tracking.
    """
    print(f"Opening input file: {input_file_path}")
    print(f"File size: {os.path.getsize(input_file_path) / (1024*1024):.2f} MB")
    
    # Open the input file and get the tree
    input_file = ROOT.TFile.Open(input_file_path, "READ")
    if not input_file or input_file.IsZombie():
        print("Error: Could not open input file!")
        return
        
    input_tree = input_file.Get("Delphes")
    if not input_tree:
        print("Error: Could not get Delphes tree!")
        input_file.Close()
        return
        
    # Print initial statistics
    print(f"Input tree entries: {input_tree.GetEntries()}")
    
    # Create output file
    output_file = ROOT.TFile(output_file_path, "RECREATE")
    if not output_file or output_file.IsZombie():
        print("Error: Could not create output file!")
        input_file.Close()
        return
    
    # Disable all branches first
    input_tree.SetBranchStatus("*", 0)
    
    # Dictionary to store enabled branches for verification
    enabled_branches = []
    
    # Enable branches with verification
    for collection in branch_list:
        print(f"\nEnabling branches for collection: {collection}")
        
        # Common branches for all collections
        common_branches = [
            f"{collection}.",
            f"{collection}_size"
        ]
        
        # Collection-specific branches
        specific_branches = {
            "Jet": [".PT", ".Eta", ".Phi", ".Mass", ".BTag", ".TauTag", ".WTag"],
            "FatJet": [".PT", ".Eta", ".Phi", ".Mass", ".BTag", ".TauTag", ".WTag"],
            "Electron": [".PT", ".Eta", ".Phi", ".Charge", ".IsolationVar"],
            "Muon": [".PT", ".Eta", ".Phi", ".Charge", ".IsolationVar"],
            "Particle": [".PT", ".Eta", ".Phi", ".PID", ".Mass", ".Status", ".Charge"],
            "Event": [".*"]
        }
        
        # Enable common branches
        for branch in common_branches:
            input_tree.SetBranchStatus(branch, 1)
            enabled_branches.append(branch)
            
        # Enable specific branches for the collection
        if collection in specific_branches:
            for suffix in specific_branches[collection]:
                branch_name = f"{collection}{suffix}"
                input_tree.SetBranchStatus(branch_name, 1)
                enabled_branches.append(branch_name)
                
    # Print enabled branches for verification
    print("\nEnabled branches:")
    for branch in enabled_branches:
        print(f"  - {branch}")
    
    # Clone the tree with progress reporting
    print("\nCloning tree...")
    output_tree = input_tree.CloneTree(-1, "fast")
    
    if not output_tree:
        print("Error: Failed to clone tree!")
        input_file.Close()
        output_file.Close()
        return
        
    print(f"Output tree entries: {output_tree.GetEntries()}")
    
    # Write and close
    print("Writing output file...")
    output_tree.Write()
    output_file.Write()
    output_file.Close()
    input_file.Close()
    
    # Verify output file
    if os.path.exists(output_file_path):
        output_size = os.path.getsize(output_file_path) / (1024*1024)
        print(f"\nOutput file created successfully:")
        print(f"Path: {output_file_path}")
        print(f"Size: {output_size:.2f} MB")
    else:
        print("Error: Output file was not created!")

# Example usage
if __name__ == "__main__":
    # Example branch list - modify according to your needs
    branches_to_keep = [
        "Event",
        "Jet",
        "Electron",
        "Muon",
        "Particle",
        "FatJet"
    ]
    
    create_mini_root_file(
        input_file_path="/home/sandeep/software/MG5_aMC_v2_9_21/pp24tops/Events/run_10/tag_1_delphes_events.root",
        output_file_path="/home/sandeep/software/MG5_aMC_v2_9_21/pp24tops/Events/run_10/PtElMu.root",
        branch_list=branches_to_keep
    )

Welcome to JupyROOT 6.30/08
Opening input file: /home/sandeep/software/MG5_aMC_v2_9_21/pp24tops/Events/run_10/tag_1_delphes_events.root
File size: 49101.94 MB


Warning in <TClass::Init>: no dictionary for class HepMCEvent is available
Warning in <TClass::Init>: no dictionary for class Event is available
Warning in <TClass::Init>: no dictionary for class Weight is available
Warning in <TClass::Init>: no dictionary for class GenParticle is available
Warning in <TClass::Init>: no dictionary for class SortableObject is available
Warning in <TClass::Init>: no dictionary for class Track is available
Warning in <TClass::Init>: no dictionary for class Tower is available
Warning in <TClass::Init>: no dictionary for class Jet is available
Warning in <TClass::Init>: no dictionary for class MissingET is available
Warning in <TClass::Init>: no dictionary for class Electron is available
Warning in <TClass::Init>: no dictionary for class Photon is available
Warning in <TClass::Init>: no dictionary for class Muon is available
Warning in <TClass::Init>: no dictionary for class ScalarHT is available
Error in <TTree::SetBranchStatus>: unknown branch -> Event.
E

Input tree entries: 475587

Enabling branches for collection: Event

Enabling branches for collection: Jet

Enabling branches for collection: Electron

Enabling branches for collection: Muon

Enabling branches for collection: Particle

Enabling branches for collection: FatJet

Enabled branches:
  - Event.
  - Event_size
  - Event.*
  - Jet.
  - Jet_size
  - Jet.PT
  - Jet.Eta
  - Jet.Phi
  - Jet.Mass
  - Jet.BTag
  - Jet.TauTag
  - Jet.WTag
  - Electron.
  - Electron_size
  - Electron.PT
  - Electron.Eta
  - Electron.Phi
  - Electron.Charge
  - Electron.IsolationVar
  - Muon.
  - Muon_size
  - Muon.PT
  - Muon.Eta
  - Muon.Phi
  - Muon.Charge
  - Muon.IsolationVar
  - Particle.
  - Particle_size
  - Particle.PT
  - Particle.Eta
  - Particle.Phi
  - Particle.PID
  - Particle.Mass
  - Particle.Status
  - Particle.Charge
  - FatJet.
  - FatJet_size
  - FatJet.PT
  - FatJet.Eta
  - FatJet.Phi
  - FatJet.Mass
  - FatJet.BTag
  - FatJet.TauTag
  - FatJet.WTag

Cloning tree...
Output tree ent